Test to pull data (extract) from yahoofinancials API (/v6/finance/quote) and put data into a dataframe.
Uses Henry's basic subscription (100 daily calls) API key obtained from yahoofinancials web site
Tested with one symbol ("DIS") then with multiple symbols ("AAPL,BTC-USD,DIS,AMZN").  Successfully able to get the data for all symbols and bring them into a new DataFrame.

Excluded: open our db and insert the DataFrame data.  Initial approach is to take all the data columns returned and insert it into the db.

In [1]:
# from requests import get
import pandas as pd

In [2]:
##  Initial attempt to loop through the stock symbols; FAILS because we need to include header and api key.
##  This is attempted correctly in the subsequent cells 

# stocks=['AAPL', 'DIS', 'UBER']
# url='https://yfapi.net/v6/finance/quote?region=US&lang=en&symbols='
# for s in stocks:
#     url_s=url + s
#     response = get(url_s)
#     print(response.json)

In [10]:
## Successful query of /v6/finance/quote API
##  Need to have your own config.py file in same folder so you can import your api key
import requests
import pprint
from config import API_KEY

url = "https://rest.yahoofinanceapi.com/v6/finance/quote"

## Example (Warning: no space between symbols):
querystring = {"symbols":"AAPL,BTC-USD,DIS,AMZN"}
# querystring = {"symbols" : "DIS"}

# use API_KEY
headers = {
    'x-api-key': API_KEY
    }

response = requests.request("GET", url, headers=headers, params=querystring).json()

# What does the response look like?  We will have to use indexing to get to the 'result' level
pprint.pprint(response)

{'quoteResponse': {'error': None,
                   'result': [{'ask': 145.8,
                               'askSize': 11,
                               'averageAnalystRating': '1.9 - Buy',
                               'averageDailyVolume10Day': 95639625,
                               'averageDailyVolume3Month': 79521745,
                               'bid': 145.75,
                               'bidSize': 14,
                               'bookValue': 3.882,
                               'currency': 'USD',
                               'displayName': 'Apple',
                               'dividendDate': 1628726400,
                               'earningsTimestamp': 1627403400,
                               'earningsTimestampEnd': 1635768000,
                               'earningsTimestampStart': 1635332340,
                               'epsCurrentYear': 5.59,
                               'epsForward': 5.71,
                               'epsTrailingTwelveMonths':

In [11]:
## How many responses did we get?
len(response['quoteResponse']['result'])

4

In [12]:
# Parse through the resulting dictionary tree to get at the 'results'.  Use this as entry(ies) to DB
response['quoteResponse']['result']

[{'language': 'en-US',
  'region': 'US',
  'quoteType': 'EQUITY',
  'quoteSourceName': 'Delayed Quote',
  'triggerable': True,
  'currency': 'USD',
  'marketState': 'CLOSED',
  'fullExchangeName': 'NasdaqGS',
  'fiftyTwoWeekLowChange': 42.96,
  'fiftyTwoWeekLowChangePercent': 0.41668284,
  'fiftyTwoWeekRange': '103.1 - 157.26',
  'fiftyTwoWeekHighChange': -11.199997,
  'fiftyTwoWeekHighChangePercent': -0.071219616,
  'fiftyTwoWeekLow': 103.1,
  'fiftyTwoWeekHigh': 157.26,
  'dividendDate': 1628726400,
  'earningsTimestamp': 1627403400,
  'earningsTimestampStart': 1635332340,
  'earningsTimestampEnd': 1635768000,
  'trailingAnnualDividendRate': 0.835,
  'trailingPE': 28.594362,
  'trailingAnnualDividendYield': 0.0056119366,
  'epsTrailingTwelveMonths': 5.108,
  'epsForward': 5.71,
  'epsCurrentYear': 5.59,
  'priceEpsCurrentYear': 26.1288,
  'sharesOutstanding': 16530199552,
  'bookValue': 3.882,
  'fiftyDayAverage': 149.21771,
  'fiftyDayAverageChange': -3.1577148,
  'fiftyDayAverageCh

In [13]:
# Create a new DataFrame from the dictionary
response_df = pd.DataFrame(response['quoteResponse']['result'])
response_df.head()

,language,region,quoteType,quoteSourceName,triggerable,currency,marketState,fullExchangeName,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,...,displayName,symbol,coinImageUrl,circulatingSupply,lastMarket,volume24Hr,volumeAllCurrencies,fromCurrency,toCurrency,startDate
0,en-US,US,EQUITY,Delayed Quote,True,USD,CLOSED,NasdaqGS,42.96,0.416683,...,Apple,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en-US,US,CRYPTOCURRENCY,CoinMarketCap,True,USD,REGULAR,CCC,37802.00,3.711254,...,NaN,BTC-USD,https://s.yimg.com/uc/fin/img/reports-thumbnai...,18819700.0,CoinMarketCap,2.822766e+10,2.822766e+10,BTC,USD=X,1.367104e+09
2,en-US,US,EQUITY,Delayed Quote,True,USD,CLOSED,NYSE,66.24,0.565043,...,NaN,DIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,en-US,US,EQUITY,Delayed Quote,True,USD,CLOSED,NasdaqGS,591.52,0.206033,...,Amazon.com,AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
